<div style="text-align:center;">
    <img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">
    <h1>CSCI 312, Fall 2025</h1>
    <h1>Effective C, Chapter 2</h1>
    <h1>Objects, functions, and types</h1>
</div>

# Python vs. C vs. C++

|           | Python          | C            | C++   | Java  |
| :--------:| :-------------: | :----------: | :---: | :---: |
| Booleans | True, False | true, false | same as C | same as C |
| ```char``` type |    | ```char``` | same as C | same as C |
| character literals |  | ```'a'```, ```'b'``` | same as C | same as C |
| Unicode characters | ```\uxxxx``` or ```\Uxxxxxxxx``` | same | same | same |

# Calling functions in C

C distinguishes between function **parameters**, which are placeholders that appear in a function declaration, and **arguments**, which are real values passed in a function call.

C is **call-by-value**: when a function is called, copies of its arguments are passed to the function.

By way of comparison, Fortran is a **call-by-reference** language: when a function is called, references (pointers) to the arguments are passed to the function.  Call-by-reference allows functions to modify objects in the calling environment.

Python is schizophrenic:
* it is call-by-value for some types of arguments (primitive types such as `int` and `float`);
* it is call-by-reference for other types of arguments (compound types such as `list` or `dict`).

In [ ]:
cat -n src/bad_swap_main.c

In [ ]:
cat -n src/bad_swap.c

In [ ]:
gcc -std=c23 src/bad_swap.c src/bad_swap_main.c

In [ ]:
./a.out

The solution is to use call-by-reference.  But how?

Solution: pass the addresses of the arguments!  These will be copied (call-by-value) but still allow access to `a` and `b` in the calling environment.

In [ ]:
cat -n src/good_swap.c

In [ ]:
gcc -std=c23 src/good_swap.c src/bad_swap_main.c

In [ ]:
./a.out

Oops!  The prototype in `bad_swap_main.c` is incorrect and allows us to pass arguments of the wrong type to our swap routine.

In [ ]:
cat -n src/good_swap_main.c

In [ ]:
gcc -std=c23 src/good_swap.c src/good_swap_main.c

In [ ]:
cat -n src/final_swap_main.c

In [ ]:
cat -n src/final_swap.c

In [ ]:
gcc -std=c23 src/final_swap.c src/final_swap_main.c

In [ ]:
./a.out

# Type inference

Starting with C23, C now has type inference, just like Python!

In [ ]:
cat -n src/auto.c

# Booleans

Until C11, C did not have a boolean type.  Instead, we made do with `0` for false and `1` for true.

C now has a first-class boolean type, `bool`.  A boolean can take one of two values, `true` and `false`.

In [ ]:
cat -n src/bool.c

In [ ]:
gcc -std=c23 src/bool.c

In [ ]:
./a.out

# Characters

C distinguishes between characters and strings.  A ```char``` is a single character, while a string is a sequence of characters.

A character is of type ```char```.  A character is delimited by single-quotes ```' '```, but the delimiting quotes are not part of the character.

In C there is a duality between characters and integers.  For instance, the ASCII character ```0``` is represented by the 8 bit (1 byte) binary number ```00110000```.  Alternatively, we can think of these bits as representing the decimal number 48.

This should be familiar from Python:
* the `ord()` function takes a one-character string as its input and returns the Unicode code point for that character;
* the `chr()` function takes an integer and returns the character corresponding to that Unicode code point.

In [ ]:
cat -n src/char.py

In [ ]:
python src/char.py

The fact that characters can be interpreted as integers means we can perform arithmetic with them:

In [ ]:
cat -n src/char.c

In [ ]:
gcc -std=c23 src/char.c

In [ ]:
./a.out

# Arrays

An **array** in C is like Python's [```array```](https://docs.python.org/3/library/array.html) or Numpy's [```ndarray```](https://numpy.org/doc/stable/user/quickstart.html).  An array is a contiguous chunk of memory set aside to hold some number of variables of the same type.

We refer to individual items in an array by index, with indexing starting at `0`.

C arrays are declared as follows:
<code>
int n[42];    /* An array that can hold 42 ints. */
float x[54];  /* An array that can hold 54 floats. */
</code>

Array sizes can be variables:

In [ ]:
cat -n src/variable_size.c

In [ ]:
gcc -std=c23 -Wall -pedantic src/variable_size.c

In [ ]:
./a.out

Unlike Python and its `len()` function, there is no way to determine the length of a C array at runtime.  You will need to keep track of it yourself.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> Losing track of array lengths is a common source of bugs. 🐞 🐞

C arrays differ from Python lists in several ways.
1.  An array is a contiguous region of memory, while the items in a list may be scattered across memory.  The contiguity turns out to be important in how we can access elements of an array.
2.  All of the items in an array must be of the same type, unlike a Python list.  This is because in order to allocate space for the array and look up array elements by index we need to know the width of the elements in bytes.  For instance, if we have an array of 4 byte integers, we know the first element occupies bytes 0 to 3, and the second element starts at an address 4 bytes past the beginning of the array.  Address arithmetic is an important part of C that we will discuss later.
3.  C does not perform checking for out-of-bounds indexes.  This is because performing such a check on each array access will slow things down. C is not alone in this; both Fortran and C++ do not perform array bounds checking.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> 😱🐞 C does not perform bounds checking on arrays.  C is perfectly happen to let you read or write past the ends of an array. 🐞😱

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"/> Reading past the end of a string can lead to subtle bugs. It can also lead to dramatic bugs. 🐞 🐞